# Cardiovascular Dataset – Preprocessing Notebook

This notebook takes the cleaned train/test splits from the EDA notebook and applies all
preprocessing steps needed to create model-ready features and reusable artifacts for the
modeling team.

**Main goals:**
- Load raw train/test splits from EDA
- Define final feature groups (numeric, categorical, binary, dropped)
- Handle missing values in a consistent, train-only way
- Apply encoding and scaling via a reusable `ColumnTransformer`
- Save the fitted preprocessing pipeline and artifacts
- Export model-ready train and test sets


### The notebook isn't runnable yet, this is a skeleton for the notbook

## 0. Imports and configuration

In [54]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import joblib

from pathlib import Path


## 1. Load train/test splits

We start from the splits created in the EDA notebook to avoid any leakage or
changes in sampling. These files contain the raw (unencoded, unscaled) features.


In [55]:
X_train = pd.read_csv("processed/X_train_raw.csv")
y_train = pd.read_csv("processed/y_train_raw.csv")

X_test = pd.read_csv("processed/X_test_raw.csv")
y_test = pd.read_csv("processed/y_test_raw.csv")

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((277968, 19), (30886, 19), (277968, 1), (30886, 1))

As a sanity check, we print out the first 5 instances of both the target variable and the training set

In [56]:
X_train.head()

,General_Health,Checkup,Exercise,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption,Age_num
0,Very Good,Within the past 2 years,Yes,No,No,No,No,No,Female,45-49,165,90.72,33.28,Yes,0,5,4,10,47.0
1,Good,Within the past year,Yes,No,No,No,"No, pre-diabetes or borderline diabetes",No,Male,18-24,183,131.54,39.33,No,4,60,30,4,21.0
2,Very Good,Within the past year,Yes,No,No,Yes,No,No,Male,30-34,188,92.99,26.32,No,12,4,30,4,32.0
3,Very Good,Within the past year,Yes,No,No,Yes,No,No,Female,70-74,178,84.82,26.83,Yes,1,30,16,0,72.0
4,Good,Within the past 2 years,Yes,No,No,Yes,No,No,Male,65-69,178,90.72,28.70,No,3,5,10,12,67.0


In [57]:
y_train.head()

,Heart_Disease
0,0
1,0
2,0
3,0
4,1


## 2. Finalize Target and Class Imbalance Strategy

In [58]:
imbalance_table = (
    y_train.value_counts()
           .to_frame("count")
           .assign(percent=lambda df: (df["count"] / df["count"].sum() * 100).round(2))
)

imbalance_table

,count,percent
Heart_Disease,,
0,255494,91.91
1,22474,8.09


## Class Imbalance Strategy

The target variable (`Heart_Disease`) shows a mild imbalance, with a positive rate of approximately
8% in the training set. This level of imbalance is typical for cardiovascular survey data and is not
severe enough to require resampling techniques.

**We intentionally avoid oversampling or undersampling**, including SMOTE or random oversampling,
because these approaches can:
- distort the true population prevalence,
- create duplicated high-risk individuals,
- reduce calibration quality (important for medical models),
- and increase the risk of overfitting.

Instead, we rely on the following approach:

**1. Maintain the original class ratio**  
Stratified train/test split (already performed in the EDA notebook) preserves real-world prevalence.

**2. Use class weights during model training**  
Models such as logistic regression, random forest, and gradient boosting will receive
`class_weight="balanced"` or an equivalent positive-to-negative scaling factor.

This strategy keeps the data distribution realistic, supports good calibration, and aligns with
best practices in clinical machine-learning pipelines.


## 3. Define feature groups

We explicitly define which columns are:
- numerical (to be scaled),
- categorical (to be one-hot encoded),
- binary (none in this dataset),
- dropped

This ensures the preprocessing pipeline is transparent and reproducible.


Let's determine which columns are binary

In [59]:
binary_cols = []

for col in X_train.columns:
    uniques = X_train[col].dropna().unique()
    if len(uniques) == 2:
        binary_cols.append(col)

binary_cols

['Exercise',
 'Skin_Cancer',
 'Other_Cancer',
 'Depression',
 'Arthritis',
 'Sex',
 'Smoking_History']

In [60]:
all_features = X_train.columns.tolist()

categorical_cols = [
    'General_Health', 'Checkup', 'Diabetes'
]

numeric_cols = [
    'Height_(cm)', 'Weight_(kg)', 'BMI',
    'Alcohol_Consumption', 'Fruit_Consumption',
    'Green_Vegetables_Consumption', 'FriedPotato_Consumption', 'Age_num'
]

binary_cols = [
    'Exercise',
    'Skin_Cancer',
    'Other_Cancer',
    'Depression',
    'Arthritis',
    'Sex',
    'Smoking_History']

We need to drop the 'Age_Category' column since we already transformed it into the feature called 'Age_num', so it would be redundant: 

In [61]:
X_train = X_train.drop(columns=["Age_Category"])
X_test  = X_test.drop(columns=["Age_Category"])

Sanity check:

In [62]:
X_train.head()

,General_Health,Checkup,Exercise,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption,Age_num
0,Very Good,Within the past 2 years,Yes,No,No,No,No,No,Female,165,90.72,33.28,Yes,0,5,4,10,47.0
1,Good,Within the past year,Yes,No,No,No,"No, pre-diabetes or borderline diabetes",No,Male,183,131.54,39.33,No,4,60,30,4,21.0
2,Very Good,Within the past year,Yes,No,No,Yes,No,No,Male,188,92.99,26.32,No,12,4,30,4,32.0
3,Very Good,Within the past year,Yes,No,No,Yes,No,No,Female,178,84.82,26.83,Yes,1,30,16,0,72.0
4,Good,Within the past 2 years,Yes,No,No,Yes,No,No,Male,178,90.72,28.70,No,3,5,10,12,67.0


In [63]:
X_test.head()

,General_Health,Checkup,Exercise,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption,Age_num
0,Very Good,Within the past year,Yes,No,No,No,No,No,Male,175,72.57,23.63,No,0,60,16,8,57.0
1,Excellent,5 or more years ago,Yes,No,No,No,No,No,Male,183,98.43,29.43,No,0,60,12,2,37.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,Female,155,76.20,31.74,No,5,30,12,8,72.0
3,Very Good,Within the past year,Yes,No,No,Yes,No,No,Male,180,104.33,32.08,Yes,1,60,60,1,27.0
4,Good,Within the past year,Yes,Yes,Yes,No,No,Yes,Male,180,92.53,28.45,Yes,0,30,16,1,77.0


## 4. Build preprocessing pipeline

We build a single `ColumnTransformer` that:

- imputes and scales numeric features,
- imputes and one-hot encodes categorical features,
- passes binary columns through unchanged.

The pipeline is fitted on the training data and then applied consistently to
both train and test sets. The fitted object is saved as an artifact.


In [64]:
numeric_steps = [
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
]

categorical_steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # ok if not huge
]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline(numeric_steps), numeric_cols),
        ("cat", Pipeline(categorical_steps), categorical_cols),
    ]
)


Preprocessing is done with a ColumnTransformer to ensure identical transformations for train and test data. One-hot encoding is used for categorical features since their cardinality is limited and does not lead to an excessive feature dimension in PyTorch.


## 5. Fit on training data and transform train/test

The preprocessing pipeline is fitted **only on `X_train`** to avoid data leakage.
The same fitted object is then used to transform both `X_train` and `X_test`.


In [65]:
# Fit on training data
X_train_pre = preprocessor.fit_transform(X_train)
X_test_pre  = preprocessor.transform(X_test)

X_train_pre.shape, X_test_pre.shape


((277968, 22), (30886, 22))

In [66]:
preprocessor.get_feature_names_out()

array(['num__Height_(cm)', 'num__Weight_(kg)', 'num__BMI',
       'num__Alcohol_Consumption', 'num__Fruit_Consumption',
       'num__Green_Vegetables_Consumption',
       'num__FriedPotato_Consumption', 'num__Age_num',
       'cat__General_Health_Excellent', 'cat__General_Health_Fair',
       'cat__General_Health_Good', 'cat__General_Health_Poor',
       'cat__General_Health_Very Good',
       'cat__Checkup_5 or more years ago', 'cat__Checkup_Never',
       'cat__Checkup_Within the past 2 years',
       'cat__Checkup_Within the past 5 years',
       'cat__Checkup_Within the past year', 'cat__Diabetes_No',
       'cat__Diabetes_No, pre-diabetes or borderline diabetes',
       'cat__Diabetes_Yes',
       'cat__Diabetes_Yes, but female told only during pregnancy'],
      dtype=object)

## 6. Sanity checks on transformed data

We run basic checks to ensure:
- No missing values remain after preprocessing
- Shapes of transformed matrices are as expected
- The class balance has not changed


In [67]:
# Check for NaNs
print("NaNs in X_train_pre:", np.isnan(X_train_pre).sum())
print("NaNs in X_test_pre:", np.isnan(X_test_pre).sum())

# Check target balance
print("Train positive rate:", y_train.mean())
print("Test positive rate:", y_test.mean())

NaNs in X_train_pre: 0
NaNs in X_test_pre: 0
Train positive rate: Heart_Disease    0.080851
dtype: float64
Test positive rate: Heart_Disease    0.080846
dtype: float64


## 7. Save preprocessing artifacts and model-ready data

We save:
- The fitted `preprocessor` (ColumnTransformer)
- Model-ready train and test matrices, along with targets

The modeling team can now load these directly without rerunning EDA.


In [69]:
y_train = y_train.squeeze("columns")
y_test  = y_test.squeeze("columns")

In [70]:
# save fitted preprocessor + feature names
joblib.dump(preprocessor, "processed/preprocessor.joblib")
joblib.dump(preprocessor.get_feature_names_out(), "processed/feature_names.joblib")

# save model-ready arrays + targets
np.savez_compressed(
    "processed/dataset_arrays.npz",
    X_train=X_train_pre,
    y_train=y_train.to_numpy(),
    X_test=X_test_pre,
    y_test=y_test.to_numpy(),
)

## 8. Summary and handoff notes

- Input: raw train/test splits from EDA notebook (`X_train_raw`, `X_test_raw`)
- Preprocessing steps:
  - Dropped columns: `age_category`
  - Numeric features: median imputation + standard scaling
  - Categorical features: most-frequent imputation + one-hot encoding

- Model-ready data:
  - `processed/preprocessor.joblib`
  - `processed/feature_names.joblib`
  - `processed/dataset_arrays.npz`

The Code is as bad as who wrote it.
